# Bob 

## Example based on the inviter_template

### Imports

In [1]:
from aries_cloudcontroller import AriesAgentController
import os
from termcolor import colored

### Initialise the Agent Controller

In [2]:
api_key = os.getenv("ACAPY_ADMIN_API_KEY")
admin_url = os.getenv("ADMIN_URL")

print(f"Initialising a controller with admin api at {admin_url} and an api key of {api_key}")
agent_controller = AriesAgentController(admin_url,api_key)

Initialising a controller with admin api at http://bob-agent:3021 and an api key of adminApiKey


### Start a Webhook Server

In [3]:
webhook_port = int(os.getenv("WEBHOOK_PORT"))
webhook_host = "0.0.0.0"

await agent_controller.init_webhook_server(webhook_host, webhook_port)

print(f"Listening for webhooks from agent at http://{webhook_host}:{webhook_port}")

Listening for webhooks from agent at http://0.0.0.0:3010


## Register Agent Event Listeners

You can see some examples within the webhook_listeners recipe. Copy any relevant cells across and customise as needed.

In [5]:
listeners = []

# Receive connection messages
def connections_handler(payload):
    state = payload['state']
    connection_id = payload["connection_id"]
    their_role = payload["their_role"]
    routing_state = payload["routing_state"]
    
    print("----------------------------------------------------------")
    print("Connection Webhook Event Received")
    print("Connection ID : ", connection_id)
    print("State : ", state)
    print("Routing State : ", routing_state)
    print("Their Role : ", their_role)
    print("----------------------------------------------------------")

    if state == "invitation":
        # Your business logic
        print("invitation")
    elif state == "request":
        # Your business logic
        print("request")

    elif state == "response":
        # Your business logic
        print("response")
    elif state == "active":
        # Your business logic
        print(colored("Connection ID: {0} is now active.".format(connection_id), "green", attrs=["bold"]))



connection_listener = {
    "handler": connections_handler,
    "topic": "connections"
}

listeners.append(connection_listener)

## YOUR LISTENERS HERE
def holder_handler(payload):
    connection_id = payload['connection_id']
    exchange_id = payload['credential_exchange_id']
    state = payload['state']
    role = payload['role']
    print("\n---------------------------------------------------\n")
    print("Handle Issue Credential Webhook")
    print(f"Connection ID : {connection_id}")
    print(f"Credential exchange ID : {exchange_id}")
    print("Agent Protocol Role : ", role)
    print("Protocol State : ", state )
    print("\n---------------------------------------------------\n")
    print("Handle Credential Webhook Payload")
    
    if state == "offer_received":
        print("Credential Offer Recieved")
        proposal = payload["credential_proposal_dict"]
        print("The proposal dictionary is likely how you would understand and display a credential offer in your application")
        print("\n", proposal)
        print("\n This includes the set of attributes you are being offered")
        attributes = proposal['credential_proposal']['attributes']
        print(attributes)
        ## YOUR LOGIC HERE
    elif state == "request_sent":
        print("\nA credential request object contains the commitment to the agents master secret using the nonce from the offer")
        ## YOUR LOGIC HERE
    elif state == "credential_received":
        print("Received Credential")
        ## YOUR LOGIC HERE
    elif state == "credential_acked":
        ## YOUR LOGIC HERE
        credential = payload["credential"]
        print("Credential Stored\n")
        print(credential)
        
        print("\nThe referent acts as the identifier for retrieving the raw credential from the wallet")
        # Note: You would probably save this in your application database
        credential_referent = credential["referent"]
        print("Referent", credential_referent)
        
holder_listener = {
    "topic": "issue_credential",
    "handler": holder_handler
}

listeners.append(holder_listener)

def prover_proof_handler(payload):
    role = payload["role"]
    connection_id = payload["connection_id"]
    pres_ex_id = payload["presentation_exchange_id"]
    state = payload["state"]
    print("\n---------------------------------------------------------------------\n")
    print("Handle present-proof")
    print("Connection ID : ", connection_id)
    print("Presentation Exchange ID : ", pres_ex_id)
    print("Protocol State : ", state)
    print("Agent Role : ", role)
    print("Initiator : ", payload["initiator"])
    print("\n---------------------------------------------------------------------\n")
    
    
    if state == "request_received":
        presentation_request = payload["presentation_request"]
        print("Recieved Presentation Request\n")
        print("\nRequested Attributes - Note the restrictions. These limit the credentials we could respond with\n")
        print(presentation_request["requested_attributes"])
    elif state == "presentation_sent":
        print("Presentation Sent\n")
        
    elif state == "presentation_acked":
        print("Presentation has been acknowledged by the Issuer")
        
prover_listener = {
    "topic": "present_proof",
    "handler": prover_proof_handler
}

listeners.append(prover_listener)

agent_controller.register_listeners(listeners)

## Create Invitation

Note the current arguments specified are in their default configurations. 

In [6]:
# Alias for invited connection
alias = None
auto_accept = False
# Use public DID?
public = "false"
# Should this invitation be usable by multiple invitees?
multi_use = "false"

invitation_response = await agent_controller.connections.create_invitation(alias, auto_accept, public, multi_use)
# Is equivalent to above. Arguments are optionally
# invitation_response = await agent_controller.connections.create_invitation()



# You probably want to keep this somewhere so you can enage in other protocols with this connection.
connection_id = invitation_response["connection_id"]


----------------------------------------------------------
Connection Webhook Event Received
Connection ID :  e1020f06-707f-4d5a-9105-fab533993cc9
State :  invitation
Routing State :  none
Their Role :  invitee
----------------------------------------------------------
invitation


## Share Invitation Object with External Agent

Typically in this jupyter notebook playground that involves copying it across to another agent's business logic notebook where they are the invitee. (see invitee_template recipe)

In [7]:
invitation = invitation_response["invitation"]
## Copy this output
print(invitation)

{'@type': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/connections/1.0/invitation', '@id': 'e743b38b-fab7-4315-92a7-1b7b6cab0f95', 'recipientKeys': ['36CZQj9EpiGbPG926PU7GrnApdqJgpGE91dKQHWHEzZS'], 'serviceEndpoint': 'https://588d5381c2e8.ngrok.io', 'label': 'Bob'}
----------------------------------------------------------
Connection Webhook Event Received
Connection ID :  e1020f06-707f-4d5a-9105-fab533993cc9
State :  request
Routing State :  none
Their Role :  invitee
----------------------------------------------------------
request
----------------------------------------------------------
Connection Webhook Event Received
Connection ID :  e1020f06-707f-4d5a-9105-fab533993cc9
State :  response
Routing State :  none
Their Role :  invitee
----------------------------------------------------------
response
----------------------------------------------------------
Connection Webhook Event Received
Connection ID :  e1020f06-707f-4d5a-9105-fab533993cc9
State :  active
Routing State :  none
Their

## Request Credential From Offer

Note: Your agent will automatically respond if ACAPY_AUTO_RESPOND_CREDENTIAL_OFFER=true flag is set in .env file of agent. Default is false.

To respond to an offer you must identify the offer using the credential_exchange_id generated for it. This is available from within the issue-credential holder handler. You could add custom logic in this loop `elif state == "request":` to handle this.

However, we will fetch the credential exchange records and **assume** this agent only has one record. Customise accordingly.


In [8]:
# thread_id=None
state = "offer_received"

records_response = await agent_controller.issuer.get_records(connection_id=connection_id, state=state)
record = records_response["results"][0]
record_id = record["credential_exchange_id"]

In [9]:
response = await agent_controller.issuer.send_request_for_record(record_id)

{'auto_offer': False,
 'credential_definition_id': 'J1y6c8mp32uBxVHkRiLM71:3:CL:216776:default',
 'updated_at': '2021-05-28 14:41:26.979185Z',
 'credential_offer': {'schema_id': 'J1y6c8mp32uBxVHkRiLM71:2:aca-acc-jupyter-plyaground:0.0.1',
  'cred_def_id': 'J1y6c8mp32uBxVHkRiLM71:3:CL:216776:default',
  'key_correctness_proof': {'c': '26568154636499869643125346823927596266547137382452507723994771661464927461630',
   'xz_cap': '292610398918993877840190318700101332699221500004089834828434691399675262094729498497626580957992178887382143522009189208396192249781874559044272999820818400645911652335865675216639340319902936489166644562503244541911200855936418869397908508304273263690541015300552335965524724199907640425885257498952774334409331711812587017581220782975306113211647277697567877810032245957706583799448858394732912645113640605480139326535579950253276279052934969975046168675356123671580039248170082187663552035085322781434364595712317776412307897699335449017148315929388840250049460327321


---------------------------------------------------

Handle Issue Credential Webhook
Connection ID : e1020f06-707f-4d5a-9105-fab533993cc9
Credential exchange ID : 8239ca76-6c91-441d-84cf-6725d7deb9ad
Agent Protocol Role :  holder
Protocol State :  request_sent

---------------------------------------------------

Handle Credential Webhook Payload

A credential request object contains the commitment to the agents master secret using the nonce from the offer

---------------------------------------------------

Handle Issue Credential Webhook
Connection ID : e1020f06-707f-4d5a-9105-fab533993cc9
Credential exchange ID : 8239ca76-6c91-441d-84cf-6725d7deb9ad
Agent Protocol Role :  holder
Protocol State :  credential_received

---------------------------------------------------

Handle Credential Webhook Payload
Received Credential

---------------------------------------------------------------------

Handle present-proof
Connection ID :  e1020f06-707f-4d5a-9105-fab533993cc9
Presentation E

## Store Received Credential

This will be done automatically if the ACAPY_AUTO_STORE_CREDENTIAL=true flag is set in the .env file for this agent. Default is false.

Again you could handle this in your holder handler function in the `elif state == "credential_received":` loop.

In [18]:
# Optionally specify an identifier to uniquely identify this credential within your agents wallet.
# You would likely want to save this somewhere.
# If not set a random one will be generated for you
credential_id = "Some Example Credential Identifier"

store_cred_response = await agent_controller.issuer.store_credential(record_id, credential_id)


---------------------------------------------------

Handle Issue Credential Webhook
Connection ID : e1020f06-707f-4d5a-9105-fab533993cc9
Credential exchange ID : 8239ca76-6c91-441d-84cf-6725d7deb9ad
Agent Protocol Role :  holder
Protocol State :  credential_acked

---------------------------------------------------

Handle Credential Webhook Payload
Credential Stored

{'referent': 'Some Example Credential Identifier', 'attrs': {'example': 'sOME DATA'}, 'schema_id': 'J1y6c8mp32uBxVHkRiLM71:2:aca-acc-jupyter-plyaground:0.0.1', 'cred_def_id': 'J1y6c8mp32uBxVHkRiLM71:3:CL:216776:default', 'rev_reg_id': None, 'cred_rev_id': None}

The referent acts as the identifier for retrieving the raw credential from the wallet
Referent Some Example Credential Identifier


## Fetch Presentation Records

Before you can present a presentation, you must identify the presentation record which you wish to respond to with a presentation. This could also be done through the present_proof listeners which have access to a presentation record in the payload.

In [19]:
# Optional Query parameters
verifier_connection_id = connection_id
thread_id=None
state = "request_received"
role = "prover"

proof_records_response = await agent_controller.proofs.get_records(verifier_connection_id, thread_id, state, role)

# We fetch the first record from the response. You may want to customise this further
presentation_record = proof_records_response["results"][0]
presentation_exchange_id = presentation_record["presentation_exchange_id"]

## Search For Available Credentials to Construct Presentation From

The presentation record can be used to query your agents wallet and return all credentials that could be used to construct valid presentation

In [21]:
# select credentials to provide for the proof
credentials = await agent_controller.proofs.get_presentation_credentials(presentation_exchange_id)
print("Credentials stored that could be used to satisfy the request. In some situations you applications may have a choice which credential to reveal\n")

attribute_by_reft = {}
revealed = {}
self_attested = {}
predicates = {}


# Note we are working on a friendlier api to abstract this away

if credentials:
    for credential in credentials:

        for attribute_reft in credential["presentation_referents"]:
            if attribute_reft not in attribute_by_reft:
                attribute_by_reft[attribute_reft] = credential

for (key, value) in attribute_by_reft.items():
    print(f"Attribute {presentation_record['presentation_request']['requested_attributes'][key]} can be satisfied by Credential with Referent -- {value['cred_info']['referent']}")
                
for attribute_reft in presentation_record["presentation_request"]["requested_attributes"]:
    if attribute_reft in attribute_by_reft:
        revealed[attribute_reft] = {
            "cred_id": attribute_by_reft[attribute_reft]["cred_info"][
                "referent"
            ],
            "revealed": True,
        }


print("\nGenerate the proof")
presentation = {
    "requested_predicates": predicates,
    "requested_attributes": revealed,
    "self_attested_attributes": self_attested,
}
print(presentation)

Credentials stored that could be used to satisfy the request. In some situations you applications may have a choice which credential to reveal

Attribute {'name': 'example', 'restrictions': [{'schema_id': 'J1y6c8mp32uBxVHkRiLM71:2:aca-acc-jupyter-plyaground:0.0.1', 'cred_def_id': 'J1y6c8mp32uBxVHkRiLM71:3:CL:216776:default'}]} can be satisfied by Credential with Referent Some Example Credential Identifier

Generate the proof
{'requested_predicates': {}, 'requested_attributes': {'0_example_uuid': {'cred_id': 'Some Example Credential Identifier', 'revealed': True}}, 'self_attested_attributes': {}}


## Send Presentation

A presentation is sent in represent to a presentation record that has previously been created.

In [22]:
presentation_response = await agent_controller.proofs.send_presentation(presentation_exchange_id, presentation)


---------------------------------------------------------------------

Handle present-proof
Connection ID :  e1020f06-707f-4d5a-9105-fab533993cc9
Presentation Exchange ID :  8834600c-d130-4138-b1f8-a0d707284e8a
Protocol State :  presentation_sent
Agent Role :  prover
Initiator :  external

---------------------------------------------------------------------

The Presentation object is a bit overwhelming. Let's look at it in detail


---------------------------------------------------------------------

Handle present-proof
Connection ID :  e1020f06-707f-4d5a-9105-fab533993cc9
Presentation Exchange ID :  8834600c-d130-4138-b1f8-a0d707284e8a
Protocol State :  presentation_acked
Agent Role :  prover
Initiator :  external

---------------------------------------------------------------------

Presentation has been acknowledged by the Issuer


## Terminate Controller

Whenever you have finished with this notebook, be sure to terminate the controller. This is especially important if your business logic runs across multiple notebooks.

In [ ]:
await agent_controller.terminate()